# Helmholtz 1D - Mock Cycle 2-level Predictions

## Goal
For the 1D Helhmholtz operator $A = -\Delta + k^2 I$, compare the quality of SVD coarsening and pointwise coarsening by the mock cycle asymptotic convergence factor. "pt" donotes pointwise coarsening.

## Discretization
* Fixed periodic domain with $n=16$ points and different $kh$ values.
* 3-point finite difference $A^h = [1, -2 + (kh)^2, 1]$. 
* Kaczmarz relaxation. For comparison, for $kh=0$ only we also look at Gauss-Seidel in lexicographic ordering.

In [97]:
import logging
import numpy as np
import helmholtz as hm
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise
import scipy.sparse
import sys
from numpy.ma.testutils import assert_array_almost_equal
from scipy.linalg import eig, norm

%load_ext autoreload
%autoreload 2

np.set_printoptions(linewidth=500, precision=2, suppress=False)
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig(stream=sys.stdout, level=logging.WARN, format="%(levelname)-8s %(message)s",
                    datefmt="%a, %d %b %Y %H:%M:%S")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mock Cycle Convergence Factor

In [81]:
def create_svd_coarsening(level):
    # Generate relaxed test matrix.
    n = level.a.shape[0]
    x = hm.solve.run.random_test_matrix((n,))
    b = np.zeros_like(x)
    x, _ = hm.solve.run.run_iterative_method(level.operator, lambda x: level.relax(x, b), x, num_sweeps=100)
    # Generate coarse variables (R) based on a window of x.
    aggregate_size = 4
    x_aggregate_t = x[:aggregate_size].transpose()
    r, s = hm.setup.coarsening.create_coarsening(x_aggregate_t, 0.1)
    #print(s, r.asarray().shape)
    #print(r.asarray())

    # Convert to sparse matrix + tile over domain.
    r_csr = r.tile(n // aggregate_size)
    return r_csr

def create_pointwise_coarsening(level):
    aggregate_size = 2
    r = hm.setup.coarsening.Coarsener(np.array([[1, 0]]))
    # Convert to sparse matrix + tile over domain.
    domain_size = level.a.shape[0]
    r_csr = r.tile(domain_size // aggregate_size)
    return r_csr

def create_average_coarsening(level):
    aggregate_size = 2
    r = hm.setup.coarsening.Coarsener(np.array([[1, 1]]))
    # Convert to sparse matrix + tile over domain.
    domain_size = level.a.shape[0]
    r_csr = r.tile(domain_size // aggregate_size)
    return r_csr

def create_coarsening(n, kh, relax, coarsening):
    a = hm.linalg.helmholtz_1d_5_point_operator(kh, n)
    if relax == "relax":
        relaxer = hm.solve.relax.KaczmarzRelaxer(a, scipy.sparse.eye(a.shape[0]))
    elif relax == "gs":
        relaxer = hm.solve.relax.GsRelaxer(a)
    else:
        raise Exception("Unsupported relaxation type {}".format(relax))
    level = hm.hierarchy.multilevel.Level.create_finest_level(a, relaxer)

    if coarsening == "svd":
        r = create_svd_coarsening(level)
    elif coarsening == "pt":
        r = create_pointwise_coarsening(level)
    elif coarsening == "avg":
        r = create_average_coarsening(level)
#        print(r.shape)
#        print(r.todense())
    else:
        raise Exception("Unsupported coarsening type {}".format(coarsening))
    return level, r

def mock_cycle_conv_factor(n, kh, relax, coarsening, num_relax_sweeps):
    level, r = create_coarsening(n, kh, relax, coarsening)
    x, conv_factor = hm.solve.run.run_iterative_method(
        level.operator,
        hm.solve.mock_cycle.MockCycle(lambda x, b: level.relax(x, r, num_relax_sweeps),
        hm.solve.run.random_test_matrix((n,), num_examples=1), 
        num_sweeps=10)
    return conv_factor

In [82]:
kh_values = np.array([0, 0.1, 0.25, 0.5, 0.8, 0.9, 1.0, 1.1, 1.2, 2 ** 0.5, 1.7, 1.8, 2.0])
nu_values = range(1, 5)
coarsening_values = ("svd", "pt", "avg")
n = 16

def conv_factor(n, coarsening):
    gs = np.array([[mock_cycle_conv_factor(n, 0, "gs", coarsening, nu) for nu in nu_values]])
    conv_factor = np.array([[mock_cycle_conv_factor(n, kh, "relax", coarsening, nu) 
                             for nu in nu_values] for kh in kh_values])
    return np.concatenate((gs, conv_factor))

def conv_factor_table(n):
    result = np.concatenate(
        tuple(conv_factor(n, coarsening) for coarsening in coarsening_values),
        axis=1)
    return pd.DataFrame(result, 
                          index=("0, GS", ) + 
                          tuple(("{:.1f}".format(kh) for kh in kh_values)),
                         columns=tuple("{}, nu={}".format(coarsening, nu) for coarsening in coarsening_values
                                      for nu in nu_values))

### n = 16

In [79]:
pd.options.display.float_format = "{:.3f}".format
result = conv_factor_table(16)
result

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4","avg, nu=1","avg, nu=2","avg, nu=3","avg, nu=4"
"0, GS",0.609,0.374,0.211,0.096,0.696,0.530,0.607,0.596,0.323,0.139,0.069,0.033
0.0,0.453,0.290,0.145,0.145,0.546,0.552,0.500,0.490,0.482,0.214,0.200,0.159
0.1,0.396,0.272,0.154,0.141,0.573,0.585,0.503,0.482,0.405,0.207,0.203,0.175
0.2,0.436,0.299,0.159,0.135,0.598,0.562,0.514,0.503,0.461,0.202,0.203,0.181
0.5,0.509,0.319,0.176,0.132,0.574,0.568,0.514,0.485,0.588,0.252,0.218,0.189
0.8,0.516,0.281,0.170,0.100,0.673,0.394,0.501,0.506,0.645,0.298,0.227,0.187
0.9,0.568,0.345,0.189,0.133,0.675,0.440,0.504,0.503,0.687,0.376,0.226,0.189
1.0,0.657,0.479,0.335,0.322,0.723,0.516,0.451,0.428,0.666,0.487,0.229,0.182
1.1,0.545,0.333,0.224,0.250,0.757,0.482,0.411,0.400,0.774,0.578,0.362,0.234
1.2,0.527,0.257,0.180,0.112,0.784,0.677,0.544,0.400,0.725,0.663,0.528,0.385


### n = 32

In [75]:
result = conv_factor_table(32)
result

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4","avg, nu=1","avg, nu=2","avg, nu=3","avg, nu=4"
"0, GS",0.642,0.386,0.230,0.102,0.696,0.642,0.591,0.578,0.331,0.139,0.086,0.031
0.0,0.523,0.257,0.165,0.146,0.572,0.560,0.515,0.515,0.535,0.210,0.204,0.192
0.1,0.538,0.284,0.181,0.145,0.563,0.600,0.518,0.513,0.549,0.213,0.191,0.189
0.2,0.539,0.300,0.181,0.149,0.594,0.592,0.507,0.486,0.555,0.225,0.207,0.197
0.5,0.590,0.295,0.155,0.130,0.630,0.547,0.499,0.492,0.554,0.262,0.222,0.222
0.8,0.646,0.298,0.185,0.126,0.648,0.489,0.524,0.535,0.582,0.405,0.225,0.258
0.9,0.657,0.411,0.082,0.071,0.675,0.450,0.495,0.522,0.656,0.415,0.220,0.262
1.0,0.726,0.412,0.249,0.144,0.753,0.544,0.459,0.483,0.667,0.474,0.303,0.245
1.1,0.594,0.350,0.222,0.270,0.747,0.569,0.410,0.457,0.702,0.566,0.372,0.264
1.2,0.523,0.269,0.178,0.157,0.788,0.660,0.559,0.443,0.706,0.685,0.516,0.383


## Conclusions
* SVD coarsening exhibits better convergence rates than pointwise. The rate improves with $\nu$, while for pointwise it is bounded by $0.5$.
* Gauss-Seidel is a better smoother for the Poisson case (as it well known).
* Almost the same efficiency is maintained for all values of $0 \leq kh \leq 2$.
* The result is independent of $n$.

### Question
For $kh = 0$, Gauss-Seidel and pointwise coarsening, the predicted 2-level convergence factor does not improve with $\nu$ and does not get below $\sim 0.5$, even though in practice we know that it does and that pointwise coarsening is a perfectly good coarsening of the Laplace operator.
* Why are we not getting a quantitative prediction with the mock cycle? (We could compare with local mode analysis to gain an insight into the slowest-to-converge component.)
* Can we trust it for other $kh$ values?

In [76]:
print(result.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} &  svd, nu=1 &  svd, nu=2 &  svd, nu=3 &  svd, nu=4 &  pt, nu=1 &  pt, nu=2 &  pt, nu=3 &  pt, nu=4 &  avg, nu=1 &  avg, nu=2 &  avg, nu=3 &  avg, nu=4 \\
\midrule
0, GS &       0.64 &       0.39 &       0.23 &       0.10 &      0.70 &      0.64 &      0.59 &      0.58 &       0.33 &       0.14 &       0.09 &       0.03 \\
0.0   &       0.52 &       0.26 &       0.16 &       0.15 &      0.57 &      0.56 &      0.51 &      0.52 &       0.54 &       0.21 &       0.20 &       0.19 \\
0.1   &       0.54 &       0.28 &       0.18 &       0.15 &      0.56 &      0.60 &      0.52 &      0.51 &       0.55 &       0.21 &       0.19 &       0.19 \\
0.2   &       0.54 &       0.30 &       0.18 &       0.15 &      0.59 &      0.59 &      0.51 &      0.49 &       0.56 &       0.22 &       0.21 &       0.20 \\
0.5   &       0.59 &       0.29 &       0.15 &       0.13 &      0.63 &      0.55 &      0.50 &      0.49 &       0.55 &       0.26 &       0.22 &   

In [83]:
level, r = create_coarsening(n, 0.5, "relax", "svd")

In [105]:
np.sort(np.real(eig(r.transpose().dot(r).todense())[0]))

array([-1.22e-16, -1.22e-16, -1.22e-16, -8.47e-17, -5.62e-17, -5.62e-17, -5.62e-17, -3.47e-17,  1.00e+00,  1.00e+00,  1.00e+00,  1.00e+00,  1.00e+00,  1.00e+00,  1.00e+00,  1.00e+00])

In [113]:
R = r.transpose().dot(r)
print(R.todense()[:4,:4])
print(np.sort(np.real(eig(R.todense())[0])))

[[ 0.64  0.41  0.09 -0.23]
 [ 0.41  0.35  0.23  0.08]
 [ 0.09  0.23  0.35  0.4 ]
 [-0.23  0.08  0.4   0.66]]
[-1.22e-16 -1.22e-16 -1.22e-16 -8.47e-17 -5.62e-17 -5.62e-17 -5.62e-17 -3.47e-17  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00]


In [139]:
norm(scipy.linalg.sqrtm(level.a.todense()).dot(R.todense()))

2.7985302773797684

In [136]:
np.sort(np.real(eig((R.transpose().dot(level.a)).dot(R).todense())[0]))

array([-2.62e+00, -1.83e+00, -1.83e+00, -7.14e-01, -3.72e-01, -1.43e-17, -8.05e-18, -8.05e-18,  2.79e-17,  2.79e-17,  6.91e-17,  6.91e-17,  9.63e-17,  9.57e-02,  9.57e-02,  2.34e-01])